# Starting model here

In [1]:
Plot = require 'itorch.Plot'

In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'cutorch'
require 'cunn'
require 'cunnx'

--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

aurora_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext2.csv'
nugget_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'
query_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv'
auroras_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_sentence_numtext2.csv'

data_file = csvigo.load({path = aurora_fn, mode = "large"})
nugget_file = csvigo.load({path = nugget_fn, mode = "large"})
query_file =  csvigo.load({path = query_fn, mode = "large"})
sent_file =  csvigo.load({path = auroras_fn, mode = "large"})

...Utils file loaded	


<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext2.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv	
<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_sentence_numtext2.csv	


<csv>	parsing done	


In [2]:
mymodel = 'bow'
rK = 100
batch_size = 500
nepochs = 1
print_every = 1
embed_dim = 10
learning_rate = 0.00001
usecuda=false

epsilon = 0.1
cuts = 4.                  --- This is the number of cuts we want
base_explore_rate = 0.1
delta = 1./(nepochs/cuts) --- Only using epsilon greedy strategy for (nepochs/cuts)% of the epochs

torch.manualSeed(420)

vocab_sized = getVocabSize(data_file)                       --- getting length of dictionary
vocab_sizeq = getVocabSize(query_file)                      --- getting length of dictionary
vocab_sizes = getVocabSize(sent_file)                      --- getting length of dictionary
vocab_size = math.max(vocab_sized, vocab_sizeq, vocab_sizes)

queries = grabNsamples(query_file, #query_file-1, nil)      --- Extracting all queries
nuggets = grabNsamples(nugget_file, #nugget_file-1, nil)    --- Extracting all samples
maxseqlend = getMaxseq(data_file)                             --- Extracting maximum sequence length
maxseqlenq = getMaxseq(query_file)                            --- Extracting maximum sequence length
maxseqlen = math.max(maxseqlenq, maxseqlend)

batch_model  = build_model(mymodel, vocab_size, embed_dim, 1, usecuda)
crit = nn.MSECriterion()

qs = queries[3]
x = data_file
K = rK
model = batch_model

use_cuda = usecuda
mxl = maxseqlen

Running BOW model	


In [3]:
if use_cuda then
  Tensor = torch.CudaTensor
  LongTensor = torch.CudaLongTensor
  crit = crit:cuda()
  print("...running on GPU")
else
  Tensor = torch.Tensor
  LongTensor = torch.LongTensor
  print("...running on CPU")
end

...running on CPU	


In [4]:
yrouge = torch.totable(torch.randn(#x))
action_list = torch.totable(torch.round(torch.rand(#x)))
preds_list = torch.totable(torch.round(torch.rand(#x)))
preds_list[1] = 0
ss_list = grabNsamples(sent_file, 1, #sent_file)
epoch = 0
minibatch = 1

In [5]:
loss = 0.                    --- Compute a new MSE loss each time
--- Reset the rougue each epoch
--- Looping over each bach of sentences for a given query
nbatches = torch.floor( #x / batch_size)
-- for minibatch = 1, nbatches do
if minibatch == 1 then          -- Need +1 to skip the first row
    nstart = 2
    nend = torch.round(batch_size * minibatch)
end
if minibatch == nbatches then 
    nstart = nend + 1
    nend = #x
end
if minibatch > 1 and minibatch < nbatches then 
    nstart = nend + 1
    nend = torch.round(batch_size * minibatch)
end

In [ ]:
nstart= nend+1
nend = nend + batch_size

In [6]:
x_ss  = geti_n(x, nstart, nend)
xout  = grabNsamples(x_ss, 1, #x_ss)     --- Extracting N samples
xs  = padZeros(xout, mxl)                 --- Padding the data by the maximum length
qs2 = padZeros({qs}, 5)
qrep = repeatQuery(qs2[1], #xs)
preds = geti_n(preds_list, nstart, nend)

sumry_list = buildKSummary(preds_list, ss_list, K)
sumry_ss = geti_n(sumry_list, nstart, nend)

summary = LongTensor(sumry_ss):t()
sentences = LongTensor(xs):t()
query = LongTensor(qrep):t()
actions = Tensor(geti_n(preds_list, nstart, nend)):resize(#xs, 1)

rscores, pscores, fscores = {}, {}, {}

yrouge = torch.totable(torch.randn(#x))
action_list = torch.totable(torch.round(torch.rand(#x)))
preds_list = torch.totable(torch.round(torch.ones(#x)))
ss_list = grabNsamples(sent_file, 1, #sent_file)

if use_cuda then
     actions =  actions:cuda()
end

myPreds = model:forward({sentences, summary, query, actions})
print(geti_n(torch.totable(myPreds), 1 , 5) )

if use_cuda then
    myPreds = myPreds:double()
end

preds = policy(myPreds, epsilon, #xs)
--- Concatenating predictions into a summary

predsummary = buildPredSummary(preds, xs, K)
--- Initializing rouge metrics at time {t-1} and save scores
r_t1 , p_t1, f_t1 = 0., 0., 0.
for i=1, #predsummary do
    --- Calculating rouge scores; Call get_i_n() to cumulatively compute rouge
    rscores[i] = rougeRecall(buildPredSummary(geti_n(preds, 1, i), geti_n(xout, 1, i)), nuggets) - r_t1
    pscores[i] = rougePrecision(buildPredSummary(geti_n(preds, 1, i), geti_n(xout, 1, i)), nuggets) - p_t1
    fscores[i] = rougeF1(buildPredSummary(geti_n(preds, 1, i), geti_n(xout, 1, i)), nuggets) - f_t1
    r_t1, p_t1, f_t1 = rscores[i], pscores[i], fscores[i]
end

labels = Tensor(fscores)

if use_cuda then
     labels = labels:cuda()
     myPreds = myPreds:cuda()
end

-- print(#myPreds, #labels)
loss = loss + crit:forward(myPreds, labels)
grads = crit:backward(myPreds, labels)
model:zeroGradParameters()
model:backward({sentences, summary, query, actions}, grads)
model:updateParameters(learning_rate)        -- Update parameters after each minibatch

yrouge = updateTable(yrouge, fscores, nstart)
preds_list = updateTable(preds_list, preds, nstart)
predsummary2 = buildPredSummary(preds_list, xs, K)

rscore = rougeRecall(predsummary2, nuggets, K)
pscore = rougePrecision(predsummary2, nuggets, K)
fscore = rougeF1(predsummary2, nuggets, K)
--         if (epoch % print_every)==0 then
perf_string = string.format(
    "Epoch %i, epsilon =%.3f, minibatch %i/%i, sum(y)/len(y) = %i/%i, {Recall = %.6f, Precision = %.6f, F1 = %.6f}", 
    epoch, epsilon, minibatch, nbatches, sumTable(preds), #preds, rscore, pscore, fscore
    )
print(perf_string)

{
  1 : 
    {
      1 : 0.32737220974448
    }
  2 : 
    {
      1 : 0.13720760825237
    }
  3 : 
    {
      1 : 0.31425874763308
    }
  4 : 
    {
      1 : 0.13559391343291
    }
  5 : 
    {
      1 : 0.31896872188639
    }
}


Epoch 0, epsilon =0.100, minibatch 1/2, sum(y)/len(y) = 499/499, {Recall = 0.838028, Precision = 0.149110, F1 = 0.253174}	


In [32]:
getLastKTokens({1,2,3,4}, 3)

{
  1 : 4
  2 : 3
  3 : 2
}


In [33]:
zero_or_x({0}, {1,2,3,4})

{
  1 : 0
}


In [70]:
function buildPredSummary2(preds, xs, K)
    local out = {}
    for i=1, #xs do
        if i == 1 then 
            out[i] = zero_or_x(preds[i], unpackZeros(xs[i]))
        else 
            --- Update it by adding xs_i and out_{i-1}
            out[i] =  getLastKTokens(tableConcat(zero_or_x(preds[i], 
                                    unpackZeros(xs[i]) ) , out[i-1]), K)
        end
    end
    return out
end

In [61]:
tst_preds = geti_n(preds_list, 1, 3)
tst_xs = geti_n(xs, 1, 3)

In [47]:
test_samples = grabNsamples(geti_n(x, 2, 4), nil, 3)

In [66]:
test_samples

{
  1 : 
    {
      1 : 21
      2 : 16
      3 : 20
    }
  2 : 
    {
      1 : 235
      2 : 218
      3 : 129
    }
  3 : 
    {
      1 : 329
      2 : 334
      3 : 332
    }
}


In [52]:
-- unpackZeros(tst_xs[1]), unpackZeros(tst_xs[2]), unpackZeros(tst_xs[3])

In [71]:
tst = buildPredSummary2(geti_n(preds_list, 2, 4), test_samples, 4)

In [76]:
tst

{
  1 : 
    {
      1 : 21
      2 : 16
      3 : 20
    }
  2 : 
    {
      1 : 20
      2 : 16
      3 : 21
      4 : 129
    }
  3 : 
    {
      1 : 129
      2 : 21
      3 : 16
      4 : 20
    }
}


In [74]:
mx = 0
for k,v in pairs(tst) do
    mx = math.max(#v, mx)
end

In [ ]:
query:t()[1]

In [ ]:
summary:t()[1]

In [ ]:
sentences:t()[1]

In [ ]:
myPreds2 = model:forward({sentences, summary, query, actions})

In [ ]:
myPreds2

In [ ]:
sent_test = LongTensor{{0, 0}, {9, 0}}
sumry_test = LongTensor{{0, 1, 3, 5}, {4, 8, 3, 0}}
query_test = LongTensor{{0, 1, 3, 5}, {4, 8, 3, 1}}
action_test = Tensor({1, 0}):resize(2,1)

In [ ]:
action_test

In [ ]:
model:forward({sent_test:t(), sumry_test:t(), query_test:t(), action_test})

In [ ]:
mod = nn.Sequential()
:add(nn.LookupTableMaskZero(5, 5))
:add(nn.SplitTable(1,2))  -- convert tensor to list of subtensors
:add(nn.Sequencer(nn.MaskZero(nn.FastLSTM(5, 3), 1)))

In [ ]:
xin = torch.LongTensor{{0, 1, 3, 5}, {0, 0, 4, 2}}:t()

In [ ]:
mod:forward(xin)